In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import sys

from tqdm import tqdm

project_dir = os.path.dirname(os.path.dirname(os.getcwd()))
if project_dir not in sys.path:
    sys.path.append(project_dir)

from utils.data import load_data
from utils.tool import prediction_summary

In [3]:
os.chdir(project_dir)
train_loader, val_loader, test_loader = load_data()
os.chdir(os.path.join(project_dir, 'baselines/Geo_LSTM'))

File1: 3BAGEmnnQ2K4zF49Dkkoxg.csv contains missing hours
File4: 4XEJFVFOS761cvyEjOYf0g.csv contains outliers
File5: 6kzhfU9xTKCUVJMz492l2g.csv contains outliers
File6: 6nBLCf6WT06TOuUExPkBtA.csv contains missing hours
File17: JQ1px-xqQx-xKh3Oa5h9nA.csv contains missing hours
File21: OfAvTbS1SiOjQo4WKSAP9g.csv contains missing hours
File24: R2ebpAblQHylOjteA-2hlQ.csv contains missing hours
File37: jDYxIP2JQL2br5aTIAR7JQ.csv contains outliers
File38: kyRUtBOTTaK7V_-dxOJTwg.csv contains outliers
File45: wSo2iRgjT36eWC4a2joWZg.csv contains outliers


# Interpolation Algorithm

In [15]:
class Geo_Layer(nn.Module):
    def __init__(self, K=4):
        super(Geo_Layer, self).__init__()
        self.K = K

    def forward(self, X):
        """
        X is a tuple of 3 tensors: (locs, readings, target_loc)
            locs: (batch_size, n_sensor, 2)
            readings: (batch_size, window, n_sensor)
            target_loc: (batch_size, 2)
        """
        locs, readings, target_loc = X
        batch_size, window, n_sensor = readings.shape

        # Compute relative distance
        Rlocs = locs - target_loc.unsqueeze(1)
        Rdists = torch.norm(Rlocs, dim=-1).squeeze(-1)   # (batch_size, n_sensor)
        indice = torch.argsort(Rdists, dim=-1)[:, :self.K]
        indice = indice.unsqueeze(1).expand(-1, window, -1)   # (batch_size, window, K
        nearby_readings = torch.gather(readings, -1, indice)   # (batch_size, window, K)

        return nearby_readings
    
class Geo_LSTM(nn.Module):
    def __init__(self, K=4, num_layers=4, hidden_size=128, fc_hidden=1024):
        super(Geo_LSTM, self).__init__()
        self.geo_layer = Geo_Layer(K)
        self.lstm = nn.LSTM(input_size=K,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True)
        self.fc = nn.Sequential(*[
            nn.Linear(hidden_size, fc_hidden),
            nn.ReLU(),
            nn.Linear(fc_hidden, 1)
        ])

    def forward(self, X):
        """
        X is a tuple of 3 tensors: (locs, readings, target_loc)
            locs: (batch_size, n_sensor, 2)
            readings: (batch_size, window, n_sensor)
            target_loc: (batch_size, 2)
        """
        nearby_readings = self.geo_layer(X)
        lstm_out, _ = self.lstm(nearby_readings)
        out = self.fc(lstm_out[:, -1, :]).squeeze(-1)
        return out

In [ ]:
epochs = 200
lr = 1e-3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Geo_LSTM()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
criterion = nn.MSELoss()

best_val_loss = 1e10
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        locs, readings, target_loc, target_reading = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
        optimizer.zero_grad()
        pred = model((locs, readings, target_loc))
        target = target_reading[:, -1]
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    scheduler.step()

    # evaluate model
    with torch.no_grad():
        model.eval()
        val_loss = 0
        for batch in val_loader:
            locs, readings, target_loc, target_reading = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device)
            pred = model((locs, readings, target_loc))
            target = target_reading[:, -1]
            loss = criterion(pred, target)
            val_loss += loss.item()
        val_loss /= len(val_loader)
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    # save the best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'geo_lstm.pth')